In [74]:
import pandas as pd
import numpy as np

In [75]:
df = pd.read_excel('./RequiredData.xlsx', sheet_name="Reservation List")
df.columns

Index(['Reservation ID', 'Reservation Creation Date (DD-MM-YYYY)',
       'Reservation Check-in Date', 'Reservation Check-out Date ',
       'Reservation Total Price ', '#Rooms', 'Cancellation Date',
       'Type\nallotment (1),  sales allotment (2), direct hotel (3)or direct agent(4)'],
      dtype='object')

In [76]:
df.columns = ['rid', 'cr_date', 'ci_date', 'co_date', 'tot_price', 'num_rooms', 'cancel_date', 'res_type']

In [77]:
df.head()

,rid,cr_date,ci_date,co_date,tot_price,num_rooms,cancel_date,res_type
0,1,2023-01-01,2023-01-05 00:00:00,2023-12-05 00:00:00,24000,10,NaN,1
1,2,2023-01-01,2023-04-05 00:00:00,2023-09-05 00:00:00,5000,5,NaN,1
2,3,2023-01-01,2023-09-08 00:00:00,2023-10-08 00:00:00,200,5,NaN,1
3,4,2023-02-01,2023-07-09 00:00:00,16/9/2023,7200,4,2023-06-08 00:00:00,1
4,5,2023-02-01,2023-09-01 00:00:00,2023-12-01 00:00:00,81400,20,NaN,2


In [78]:
def convert_to_datetime(df, inplace=True):
    if not inplace:
        df = df.copy()
    for column in df.columns:
        if 'date' in column:
            df[column] = pd.to_datetime(df[column])
    if inplace:
        return
    else:
        return df

In [79]:
convert_to_datetime(df, inplace=True)

In [80]:
df.sort_values('cr_date', inplace=True)

In [81]:
df.head()

,rid,cr_date,ci_date,co_date,tot_price,num_rooms,cancel_date,res_type
0,1,2023-01-01,2023-01-05,2023-12-05,24000,10,NaT,1
1,2,2023-01-01,2023-04-05,2023-09-05,5000,5,NaT,1
2,3,2023-01-01,2023-09-08,2023-10-08,200,5,NaT,1
3,4,2023-02-01,2023-07-09,2023-09-16,7200,4,2023-06-08,1
4,5,2023-02-01,2023-09-01,2023-12-01,81400,20,NaT,2


In [82]:
def get_cancel_dates(df):
    cancel_dates_to_res_total_and_pay_amount = {}
    for index, row in df.iterrows():
        
        cancel_dt = row['cancel_date']
        if cancel_dt is pd.NaT:
            continue
        if cancel_dt not in cancel_dates_to_res_total_and_pay_amount.keys():
            cancel_dates_to_res_total_and_pay_amount[cancel_dt] = np.array([0, 0]) # res_total_price and numRooms
        cancel_dates_to_res_total_and_pay_amount[cancel_dt] += np.array([row['tot_price'], row['num_rooms']])

    return cancel_dates_to_res_total_and_pay_amount

In [83]:
get_cancel_dates(df)

{Timestamp('2023-06-08 00:00:00'): array([7200,    4]),
 Timestamp('2023-04-20 00:00:00'): array([74243,    55])}

In [103]:
def get_cf_df_from_res(df, res_type='all', date_index=True):
    df = df.copy()
    if res_type != 'all':
        df = df[df['res_type']==int(res_type)]
    df = df.groupby('cr_date').sum(numeric_only=True)
    df = df[['tot_price', 'num_rooms']]
    if not date_index:
        df = df.reset_index()
    return df

In [ ]:
def get_cf_df_from_pay(df, res_type='all', date_index=True):
    df = df.copy()
    df = df.groupby('pay_date').sum(numeric_only=True)
    df = df[['pay_amount']]
    if not date_index:
        df = df.reset_index()
    return df

In [106]:
cf_df_before_cancel = get_cf_df_from_res(df, res_type=1, date_index=True)

,tot_price,num_rooms
cr_date,,
2023-01-01,29200,20
2023-02-01,7200,4
2023-03-01,74243,55


In [ ]:
cf_df = 